In [ ]:
# Import required libraries
import os
import gc
import sys
import json
import random
from pathlib import Path

import cv2 # CV2 for image manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm

from imgaug import augmenters as iaa

import seaborn as sns
import matplotlib.image as mpimg
from matplotlib import pyplot as plt

from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
!pip install tensorflow==1.5
!pip install keras==2.1.5
!pip install mediapipe

import tensorflow
print(tensorflow.__version__)
import keras
print(keras.__version__)

In [ ]:
!ls /kaggle/input/imaterialist-fashion-2020-fgvc7/

In [ ]:
%%time
with open('/kaggle/input/imaterialist-fashion-2020-fgvc7/label_descriptions.json', 'r') as file:
    label_desc = json.load(file)
sample_sub_df = pd.read_csv('/kaggle/input/imaterialist-fashion-2020-fgvc7/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/imaterialist-fashion-2020-fgvc7/train.csv')

In [ ]:
train_df.head()

In [ ]:
sample_sub_df.head()

In [ ]:
print(f'Shape of training dataset: {train_df.shape}')

In [ ]:
print(f'# of images in training set: {train_df["ImageId"].nunique()}')
print(f'# of images in test set: {sample_sub_df["ImageId"].nunique()}')

### Image size analysis in training dataset

In [ ]:
pd.DataFrame([train_df['Height'].describe(), train_df['Width'].describe()]).T.loc[['max', 'min', 'mean']]

### Height and Width destribution of training images

In [ ]:
image_shape_df = train_df.groupby("ImageId")["Height", "Width"].first()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
ax1.hist(image_shape_df['Height'], bins=100)
ax1.set_title("Height distribution")
ax2.hist(image_shape_df['Width'], bins=100)
ax2.set_title("Width distribution")
plt.show()

### Image with minimum height

In [ ]:
plt.figure(figsize = (70,7))
min_height = list(set(train_df[train_df['Height'] == train_df['Height'].min()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{min_height}.jpg'))
plt.grid(False)
plt.show()

### Image with maximum height

In [ ]:
plt.figure(figsize = (70,7))
max_height = list(set(train_df[train_df['Height'] == train_df['Height'].max()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{max_height}.jpg'))
plt.grid(False)
plt.show()

### Image with minimum width

In [ ]:
plt.figure(figsize = (70,7))
min_width = list(set(train_df[train_df['Width'] == train_df['Width'].min()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{min_width}.jpg'))
plt.grid(False)
plt.show()

### Image with maximum width

In [ ]:
plt.figure(figsize = (70,7))
max_width = list(set(train_df[train_df['Width'] == train_df['Width'].max()]['ImageId']))[0]
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{max_width}.jpg'))
plt.grid(False)
plt.show()

In [ ]:
area_df = pd.DataFrame()
area_df['ImageId'] = train_df['ImageId']
area_df['area'] = train_df['Height'] * train_df['Width']
min_area = list(set(area_df[area_df['area'] == area_df['area'].min()]['ImageId']))[0]
max_area = list(set(area_df[area_df['area'] == area_df['area'].max()]['ImageId']))[0]

### Image with minimum area

In [ ]:
plt.figure(figsize = (70,7))
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{min_area}.jpg'))
plt.grid(False)
plt.show()

### Image with maximum area

In [ ]:
plt.figure(figsize = (70,7))
plt.imshow(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{max_area}.jpg'))
plt.grid(False)
plt.show()

## Details about Classes and Attributes

In [ ]:
num_classes = len(label_desc['categories'])
num_attributes = len(label_desc['attributes'])
print(f'Total # of classes: {num_classes}')
print(f'Total # of attributes: {num_attributes}')

In [ ]:
categories_df = pd.DataFrame(label_desc['categories'])
attributes_df = pd.DataFrame(label_desc['attributes'])
# categories_df

In [ ]:
pd.set_option('display.max_rows', 300)
# attributes_df

## Plotting a few training images without any masks

In [ ]:
def plot_images(size=12, figsize=(12, 12)):
    # First get some images to be plotted
    image_ids = train_df['ImageId'].unique()[:12]
    images=[]
    
    for image in image_ids:
        images.append(mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{image}.jpg'))
    
    # Plot images in groups of 4 images
    n_groups = 4
    
    count = 0
    for index in range(size // 4):
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
        for row in ax:
            for col in row:
                col.imshow(images[count])
                col.axis('off')
                count += 1
        plt.show()
    gc.collect()

In [ ]:
plot_images()

## Plotting a few images with given segments

In [ ]:
def create_mask(size):
    image_ids = train_df['ImageId'].unique()[:size]
    images_meta=[]

    for image_id in image_ids:
        img = mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{image_id}.jpg')
        images_meta.append({
            'image': img,
            'shape': img.shape,
            'encoded_pixels': train_df[train_df['ImageId'] == image_id]['EncodedPixels'],
            'class_ids':  train_df[train_df['ImageId'] == image_id]['ClassId']
        })

    masks = []
    for image in images_meta:
        shape = image.get('shape')
        encoded_pixels = list(image.get('encoded_pixels'))
        class_ids = list(image.get('class_ids'))
        
        # Initialize numpy array with shape same as image size
        height, width = shape[:2]
        mask = np.zeros((height, width)).reshape(-1)
        
        # Iterate over encoded pixels and create mask
        for segment, (pixel_str, class_id) in enumerate(zip(encoded_pixels, class_ids)):
            splitted_pixels = list(map(int, pixel_str.split()))
            pixel_starts = splitted_pixels[::2]
            run_lengths = splitted_pixels[1::2]
            assert max(pixel_starts) < mask.shape[0]
            for pixel_start, run_length in zip(pixel_starts, run_lengths):
                pixel_start = int(pixel_start) - 1
                run_length = int(run_length)
                mask[pixel_start:pixel_start+run_length] = 255 - class_id * 4
        masks.append(mask.reshape((height, width), order='F'))  # https://stackoverflow.com/questions/45973722/how-does-numpy-reshape-with-order-f-work
    return masks, images_meta

In [ ]:
def plot_segmented_images(size=12, figsize=(14, 14)):
    # First create masks from given segments
    masks, images_meta = create_mask(size)
    
    # Plot images in groups of 4 images
    n_groups = 4
    
    count = 0
    for index in range(size // 4):
        fig, ax = plt.subplots(nrows=2, ncols=2, figsize=figsize)
        for row in ax:
            for col in row:
                col.imshow(images_meta[count]['image'])
                col.imshow(masks[count], alpha=0.75)
                col.axis('off')
                count += 1
#         print(index, count)
        plt.show()
    gc.collect()

In [ ]:
plot_segmented_images()

## Analysing Categories and Attributes

In [ ]:
categories_df = pd.DataFrame(label_desc.get('categories'))
attributes_df = pd.DataFrame(label_desc.get('attributes'))

In [ ]:
print(f'# of categories: {len(categories_df)}')
print(f'# of attributes: {len(attributes_df)}')

So there are 46 categories (classes) and 294 attributes. Let's see some of the categories and attributes

In [ ]:
categories_df.head()

In [ ]:
attributes_df.head()

In [ ]:
category_map, attribute_map = {}, {}
for cat in label_desc.get('categories'):
    category_map[cat.get('id')] = cat.get('name')
for attr in label_desc.get('attributes'):
    attribute_map[attr.get('id')] = attr.get('name')

In [ ]:
train_df['ClassId'] = train_df['ClassId'].map(category_map)
train_df['ClassId'] = train_df['ClassId'].astype('category')

### Let's see the class wise distribution of segments in training dataset

In [ ]:
sns.set(style='darkgrid')
fig, ax = plt.subplots(figsize = (10,10))
sns.countplot(y='ClassId',data=train_df , ax=ax, order = train_df['ClassId'].value_counts().index)
fig.show()

### Now let's visualize an image with all its classes and attributes

In [ ]:
IMAGE_ID = '000b3ec2c6eaffb491a5abb72c2e3e26'

In [ ]:
# Get the an image id given in the training set for visualization
vis_df = train_df[train_df['ImageId'] == IMAGE_ID]
vis_df['ClassId'] = vis_df['ClassId'].cat.codes
vis_df = vis_df.reset_index(drop=True)
vis_df

From above table, this image has 8 segmentes and a few attributes. Let's visualize all of them!

## Let's first the plot the plain image

In [ ]:
plt.figure(figsize = (110,11))
image = mpimg.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{IMAGE_ID}.jpg')
plt.grid(False)
plt.imshow(image)
plt.plot()

In [ ]:
train_df[train_df['ImageId'] == IMAGE_ID]

## Now let's plot each segment in a separate image

In [ ]:
segments = list(vis_df['EncodedPixels'])
class_ids = list(vis_df['ClassId'])
masks = []
for segment, class_id in zip(segments, class_ids):
    
    height = vis_df['Height'][0]
    width = vis_df['Width'][0]
    # Initialize empty mask
    mask = np.zeros((height, width)).reshape(-1)
    
    # Iterate over encoded pixels and create mask
    splitted_pixels = list(map(int, segment.split()))
    pixel_starts = splitted_pixels[::2]
    run_lengths = splitted_pixels[1::2]
    assert max(pixel_starts) < mask.shape[0]
    for pixel_start, run_length in zip(pixel_starts, run_lengths):
        pixel_start = int(pixel_start) - 1
        run_length = int(run_length)
        mask[pixel_start:pixel_start+run_length] = 255 - class_id * 4

    mask = mask.reshape((height, width), order='F')
    masks.append(mask)

In [ ]:
def plot_individual_segment(*masks, image, figsize=(110, 11)):
    plt.figure(figsize = figsize)
    plt.imshow(image)
    for mask in masks:
        plt.imshow(mask, alpha=0.6)
    plt.axis('off')
    plt.show()

## Plotting 1st Segment: ClassId: "Shoe" and no attributes 

In [ ]:
plot_individual_segment(masks[0], image=image)

## Plotting 2nd Segment: ClassId: "shoe"

In [ ]:
plot_individual_segment(masks[1], image=image)

## Plotting 3rd Segment with ClassId: "pants"

In [ ]:
plot_individual_segment(masks[2], image=image)

## Plotting 4th Segment with ClassId: "top, t-shirt, sweatshirt"

In [ ]:
plot_individual_segment(masks[3], image=image)

## Plotting 5th Segment with ClassId: "pocket"

In [ ]:
plot_individual_segment(masks[4], image=image)

## Plotting 6th Segment with ClassId: "sleeve"

In [ ]:
plot_individual_segment(masks[5], image=image)

## Plotting 7th Segment with ClassId: "sleeve"

In [ ]:
plot_individual_segment(masks[6], image=image)

## Plotting 8th segment with Class "neckline"

In [ ]:
plot_individual_segment(masks[6], image=image)

Some of the segments have no attributes. Let's check how many such segment exists in training dataset.

Let's check of missing values in training dataset for columns other than "AttributeIds"

In [ ]:
train_df[['ImageId', 'EncodedPixels', 'Height', 'Width', 'ClassId']].isna().sum()

## Data Preparation and modeling

In [ ]:
train_df.head()

In [ ]:
train_df['ClassId'] = train_df['ClassId'].cat.codes

In [ ]:
train_df

## Calculating Energy

In [ ]:
import cv2
import numpy as np

def image_energy(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    dy = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    energy = np.sqrt(np.square(dx) + np.square(dy))
    energy = np.uint8(255 * energy / np.max(energy))
    return energy

In [ ]:
plt.figure(figsize = (110,11))
unique_id = train_df.groupby('ImageId')
image_ids = train_df['ImageId'].unique()
energy = []

for id_ in image_ids:
    image = cv2.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{id_}.jpg')
    energy.append(image_energy(image))


In [ ]:
print(len(energy))
unique_id.shape

## Pose Estimation

In [ ]:
import matplotlib.pyplot as plt
from time import time
import mediapipe as mp
import cv2 as cv

In [ ]:
mpPose = mp.solutions.pose

In [ ]:
pose = mpPose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5
)

In [ ]:
mpDraw = mp.solutions.drawing_utils

In [ ]:
from collections import defaultdict

In [ ]:
len(train_df['ImageId'].unique())

In [ ]:
SIZE = defaultdict(dict)

# def detectPose(image_ids[i], masks[i], images_meta[i]['image'], pose, display=True):
def detectPose(id, masks, image, pose, display=True):
    img = cv.imread(f'/kaggle/input/imaterialist-fashion-2020-fgvc7/train/{id}.jpg')
    OutputImage = img.copy()
    imageRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    results = pose.process(img)
    imgHeight, imgWidth, _ = img.shape
    landmarks = []

    if results.pose_landmarks:
        mpDraw.draw_landmarks(
            image=OutputImage,
            landmark_list=results.pose_landmarks,
            connections=mpPose.POSE_CONNECTIONS
        )
#         print(len(mpPose.PoseLandmark))
        for i in range(len(mpPose.PoseLandmark)):
            SIZE[id][mpPose.PoseLandmark(i).name] = [results.pose_landmarks.landmark[mpPose.PoseLandmark(i).value].x * imgWidth,
                                                     results.pose_landmarks.landmark[mpPose.PoseLandmark(i).value].y * imgHeight]
      
    if display:
        
        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))
        
        ax1.imshow(img[:, :, ::-1])
        ax1.set_title('Input')
        
        ax3.axis('off')
        ax3.imshow(OutputImage[:, :, ::-1])
        ax3.set_title('Output')
        
        for i in range(4):
            ax2.axis('off')
            ax2.imshow(image[:, :, ::-1])
            ax2.imshow(masks, alpha=0.6)
            ax2.set_title('Segmentation')
    
# '''
    
n = 100
image_ids = train_df['ImageId'].unique()[:n]
masks, images_meta = create_mask(len(image_ids))
# print(len(masks), len(images_meta))
for i in range(len(image_ids)):
    if i % 10 == 0:
        print(i)
#     detectPose(image_ids[i], pose, display=True)
    detectPose(image_ids[i], masks[i], images_meta[i]['image'], pose, display=False)
# '''


In [ ]:
LANDMARKS = pd.DataFrame(SIZE).transpose()
LANDMARKS.head()

In [ ]:
list(LANDMARKS.columns)

In [ ]:
LANDMARKS = LANDMARKS[['NOSE','LEFT_EYE','RIGHT_EYE', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW', 'RIGHT_ELBOW','LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE', 'RIGHT_ANKLE']]
LANDMARKS.head()

In [ ]:
LANDMARKS.to_csv('landmarks.csv', index=False)

In [ ]:
LANDMARKS = LANDMARKS.reset_index()
LANDMARKS.head()

In [ ]:
import math

In [ ]:
p1 = [2921.795607328415, 2055.795118510723]
p2 = [1502.254955291748, 1945.0051828622818]
math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2))

In [ ]:
SIZES = {}

SIZES['SHOULDER'] = []
for (p1, p2) in zip(LANDMARKS['LEFT_SHOULDER'], LANDMARKS['RIGHT_SHOULDER']):
    SIZES['SHOULDER'].append(math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)))
    
SIZES['HIP'] = []
for (p1, p2) in zip(LANDMARKS['LEFT_HIP'], LANDMARKS['RIGHT_HIP']):
    SIZES['HIP'].append(math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)))

SIZES['LEFT_ARM'] = []
for (p1, p2, p3) in zip(LANDMARKS['LEFT_SHOULDER'], LANDMARKS['LEFT_ELBOW'], LANDMARKS['LEFT_WRIST']):
    SIZES['LEFT_ARM'].append(math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)) + math.sqrt(((p3[0]-p2[0])**2)+((p3[1]-p2[1])**2)))

SIZES['RIGHT_ARM'] = []
for (p1, p2, p3) in zip(LANDMARKS['RIGHT_SHOULDER'], LANDMARKS['RIGHT_ELBOW'], LANDMARKS['RIGHT_WRIST']):
    SIZES['RIGHT_ARM'].append(math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)) + math.sqrt(((p3[0]-p2[0])**2)+((p3[1]-p2[1])**2)))

SIZES['LEFT_LEG'] = []
for (p1, p2, p3) in zip(LANDMARKS['LEFT_HIP'], LANDMARKS['LEFT_KNEE'], LANDMARKS['LEFT_ANKLE']):
    SIZES['LEFT_LEG'].append(math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)) + math.sqrt(((p3[0]-p2[0])**2)+((p3[1]-p2[1])**2)))

SIZES['RIGHT_LEG'] = []
for (p1, p2, p3) in zip(LANDMARKS['RIGHT_HIP'], LANDMARKS['RIGHT_KNEE'], LANDMARKS['RIGHT_ANKLE']):
    SIZES['RIGHT_LEG'].append(math.sqrt(((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2)) + math.sqrt(((p3[0]-p2[0])**2)+((p3[1]-p2[1])**2)))

SIZES['HEIGHT'] = []
# SIZES['TORSO'] = []
for (p1, p2, p8, p9) in zip(LANDMARKS['LEFT_EYE'], LANDMARKS['RIGHT_EYE'], LANDMARKS['LEFT_ANKLE'], LANDMARKS['RIGHT_ANKLE']):
    eye = [(p1[0]+p2[0])/2, (p1[1]+p2[1])/2]
    ankle = [(p8[0]+p9[0])/2, (p8[1]+p9[1])/2]
    
#     head = 1.75 * math.sqrt(((p1[0]-sh[0])**2)+((p1[1]-sh[1])**2))
#     torso = math.sqrt(((hip[0]-sh[0])**2)+((hip[1]-sh[1])**2))
    
#     SIZES['TORSO'].append(torso)
    
#     thigh = math.sqrt(((hip[0]-knee[0])**2)+((hip[1]-knee[1])**2))
#     stockings = math.sqrt(((ankle[0]-knee[0])**2)+((ankle[1]-knee[1])**2))
    
    SIZES['HEIGHT'].append(math.sqrt(((ankle[0]-eye[0])**2)+((ankle[1]-eye[1])**2)))


# for (left_hip, left_ankle, right_hip, right_ankle, s_l, s_r) in zip(LANDMARKS['LEFT_HIP'], LANDMARKS['LEFT_ANKLE'], LANDMARKS['RIGHT_HIP'], LANDMARKS['RIGHT_ANKLE'], LANDMARKS['LEFT_SHOULDER'], LANDMARKS['RIGHT_SHOULDER']):
#     avg_leg_length = (math.sqrt((left_hip[0] - left_ankle[0])**2 + (left_hip[1] - left_ankle[1])**2) + 
#                       math.sqrt((right_hip[0] - right_ankle[0])**2 + (right_hip[1] - right_ankle[1])**2)) / 2
    
#     shoulder = [(s_l[0] + s_r[0]) / 2, (s_l[1] + s_r[1]) / 2]
    # Calculate the height of the person based on the average leg length and the vertical distance from the ankle to the shoulder
#     SIZES['HEIGHT'].append(avg_leg_length * (shoulder[1] - ((left_ankle[1]+right_ankle[1])/2)) / (left_ankle[1] - right_ankle[1] + 2))

SIZES = pd.DataFrame(SIZES)
SIZES.head()

In [ ]:
def get_body_size(torso_length, arm_length, leg_length, height):
    if torso_length > leg_length:
        if arm_length > height / 2:
            return "L"
        else:
            return "M"
        
    return "S"

In [ ]:
# BODY = []
# for tl, l_a, r_a, l_l, r_l, h in zip(SIZES['TORSO'], SIZES['LEFT_ARM'], SIZES['RIGHT_ARM'], SIZES['LEFT_LEG'], SIZES['RIGHT_LEG'], SIZES['HEIGHT']):
#     BODY.append(get_body_size(tl, (l_a+r_a)/2, (l_l+r_l)/2, h))

# SIZES['BODY_SIZE1'] = BODY
# SIZES

In [ ]:
# SIZES['BODY_SIZE1'].value_counts()

In [ ]:
def estimate_body_size(right_ankle, left_ankle, right_hip, left_hip, right_shoulder, left_shoulder, right_elbow, left_elbow, right_knee, left_knee, right_wrist, left_wrist, height_pixels):
    # Calculate height in inches based on pixel height
    height_inches = height_pixels / 10 # assuming 10 pixels per inch
    
    # Calculate waist and hip measurements as averages of left and right sides
    waist = abs(right_hip[0] - left_hip[0]) / 2
#     print(waist)
    hips = (right_hip[1] - left_hip[1]) / 2
    
    # Calculate bust and chest measurements as averages of left and right sides
    bust = (right_shoulder[1] - left_shoulder[1]) / 2
    chest = (right_elbow[1] - left_elbow[1]) / 2
    
    # Calculate sleeve length as average of left and right sides
    sleeve_length = ((right_wrist[1] - right_shoulder[1]) + (left_wrist[1] - left_shoulder[1])) / 2
    
    # Calculate pant length as average of left and right legs
    pant_length = ((right_ankle[1] - right_hip[1]) + (left_ankle[1] - left_hip[1])) / 2
    
    # Use measurements to estimate body size label
    extra = 0
    h = 0
    size = "large"
    if height_inches < 75 + h:
        return height_inches, waist, "small"
#         if waist < 27 + extra:
#             size = "extra small"
        if waist < 29 + extra:
            size = "small"
        elif waist < 31 + extra:
            size = "medium"
        elif waist < 33 + extra:
            size = "large"
#         else:
#             size = "extra large"
    elif height_inches < 150 + h:
        return height_inches, waist, "medium"
#         if waist < 29 + extra:
#             size = "extra small"
        if waist < 31 + extra:
            size = "small"
        elif waist < 33 + extra:
            size = "medium"
        elif waist < 35 + extra:
            size = "large"
#         else:
#             size = "extra large"
    elif height_inches > 150 + h:
        return height_inches, waist, "large"
#         if waist < 35 + extra:
#             size = "extra small"
        if waist < 37 + extra:
            size = "small"
        elif waist < 39 + extra:
            size = "medium"
        elif waist < 41 + extra:
            size = "large"
#         else:
#             size = "extra large"

    return height_inches, waist, size

In [ ]:
'''
    elif height_inches < 68 + h:
#         if waist < 31 + extra:
#             size = "extra small"
        if waist < 33 + extra:
            size = "small"
        elif waist < 35 + extra:
            size = "medium"
        elif waist < 37 + extra:
            size = "large"
#         else:
#             size = "extra large"
    elif height_inches < 72 + h:
#         if waist < 33 + extra:
#             size = "extra small"
        if waist < 35 + extra:
            size = "small"
        elif waist < 37 + extra:
            size = "medium"
        elif waist < 39 + extra:
            size = "large"
#         else:
#             size = "extra large"
'''

In [ ]:
BODY = []
H = []
W = []
for (right_ankle, left_ankle, right_hip, left_hip, right_shoulder, left_shoulder, right_elbow, 
    left_elbow, right_knee, left_knee, right_wrist, left_wrist, height_pixels) in zip(LANDMARKS['RIGHT_ANKLE'], LANDMARKS['LEFT_ANKLE'], LANDMARKS['RIGHT_HIP'], LANDMARKS['LEFT_HIP'], 
                                                                                     LANDMARKS['RIGHT_SHOULDER'], LANDMARKS['LEFT_SHOULDER'], LANDMARKS['RIGHT_ELBOW'], LANDMARKS['LEFT_ELBOW'], 
                                                                                     LANDMARKS['RIGHT_KNEE'], LANDMARKS['LEFT_KNEE'], LANDMARKS['RIGHT_WRIST'], LANDMARKS['LEFT_WRIST'], SIZES['HEIGHT']):
    h, w, s = estimate_body_size(right_ankle, left_ankle, right_hip, left_hip, right_shoulder, left_shoulder, right_elbow, left_elbow, right_knee, left_knee, right_wrist, left_wrist, height_pixels)
    BODY.append(s)
    H.append(h)
    W.append(w)
    
SIZES['BODY_SIZE2'] = BODY
SIZES['HEIGHT'] = H
SIZES['WAIST'] = W
SIZES['BODY_SIZE2'].value_counts()

In [ ]:
SIZES['BODY_SIZE2'].value_counts()

In [ ]:
sns.set(style='darkgrid')
fig, ax = plt.subplots(figsize = (10,10))
sns.countplot(y='BODY_SIZE2',data=SIZES , ax=ax, order = SIZES['BODY_SIZE2'].value_counts().index)
fig.show()

In [ ]:
SIZES.head()

In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile app.py
import streamlit as st

x = st.slider('Select a value')
st.write(x, 'squared is', x * x)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'
        
!streamlit run /kaggle/working/app.py

In [ ]:
!npm install localtunnel

!streamlit run /kaggle/working/app.py

!npx localtunnel --port 8501

In [ ]:
# fig = plt.figure(figsize=(20,7))

# for h, w, s in zip(SIZES['HEIGHT'], SIZES['WAIST'], SIZES['BODY_SIZE2']):
#     if s == "large":
#         plt.scatter(h, w, color='r')
#     elif s == "small":
#         plt.scatter(h, w, color='g')
#     else:
#         plt.scatter(h, w, color='b')

# #     plt.legend(loc='best')
# #     plt.title(title)
# plt.xlabel("Height")
# plt.ylabel("Waist")
# plt.show()